In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords 

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
stop = stopwords.words('english')

In [5]:
df = pd.read_csv('/content/drive/My Drive/SI650/model_final.csv')

In [6]:
lowered_summary = [str(k).lower() for k in df.summary]
df['summary'] = lowered_summary

In [7]:
replaced = []
for i in df.summary:
  i = re.sub(r"[^ a-zA-Z0-9]+", "", i)
  replaced.append(i)

In [8]:
df['summary'] = replaced

In [9]:
splitted = [k.split() for k in df.summary]

In [10]:
new_words = []
for k in splitted:
  instance = []
  for l in k:
    if l not in stop:
      instance.append(l)
  new_words.append(instance)

In [11]:
joined = []
for i in new_words:
  sentences = " ".join(i)
  joined.append(sentences)

In [12]:
df['summary'] = joined
df

,Unnamed: 0,ID,Content,summary,Title,Organization,Date,URL
0,0,0,NaN,health promotion glossary world health organiz...,Health Promotion Glossary,IFRC,1998-01-06T11:10:08Z,https://www.who.int/healthpromotion/about/HPR%...
1,1,1,Microsoft Word - health-policy.doc Health...,activities support health always central carry...,Microsoft Word - health-policy.doc,IFRC,2006-10-20T11:53:20Z,https://www.ifrc.org/Global/Publications/Healt...
2,2,2,NaN,schools wideranging research programmes examin...,NaN,IFRC,2000-12-06T13:57:01Z,https://www.ircwash.org/sites/default/files/Be...
3,4,4,NaN,xiii written xiv guide organizedxv 1 purpose g...,Improving Health through Behavior Change A Pro...,IFRC,2004-08-27T18:49:28Z,http://www.ehproject.org/PDF/Joint_Publication...
4,5,5,"www.ifrc.org Saving lives, changing minds. vo...",40 million blood donations red cross red cresc...,Voluntary blood donation,IFRC,2012-12-04T08:08:25Z,https://www.ifrc.org/PageFiles/53503/1226403-I...
...,...,...,...,...,...,...,...,...
212,234,234,NaN,act disasters health emergencies meet needs im...,World Disasters Report 2018,IFRC,2018-11-27T09:22:32Z,http://media.ifrc.org/ifrc/wp-content/uploads/...
213,235,235,"Climate change, water and food security - ODI...",odi background notes provide summary snapshot ...,"Climate change, water and food security - ODI",IFRC,2009-03-11T09:04:07Z,https://www.odi.org/sites/odi.org.uk/files/odi...
214,236,236,Food Security and Nutrition Indicator Basics ...,level hunger decreased 27 millions people stil...,Food Security and Nutrition Indicator Basics,IFRC,2017-11-29T13:20:33Z,https://ifrcwatsanmissionassistant.files.wordp...
215,237,237,NaN,ifrs group report new structure kpmg sa geneva...,IFRC Annual Report 2017,IFRC,2018-04-18T07:02:36Z,https://media.ifrc.org/ifrc/wp-content/uploads...


In [13]:
path = r'/content/drive/My Drive/SI650/GoogleNews-vectors-negative300-SLIM.bin'

In [14]:
import gensim
import numpy as np

In [15]:
w2vec = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)

In [16]:
def get_embeddings(word):
    if word in w2vec.vocab:
        return w2vec[word]
    else:
        return np.zeros(300)

In [17]:
out_dict = {}

In [18]:
df_vector = df[['ID','summary']]
df_vector

,ID,summary
0,0,health promotion glossary world health organiz...
1,1,activities support health always central carry...
2,2,schools wideranging research programmes examin...
3,4,xiii written xiv guide organizedxv 1 purpose g...
4,5,40 million blood donations red cross red cresc...
...,...,...
212,234,act disasters health emergencies meet needs im...
213,235,odi background notes provide summary snapshot ...
214,236,level hunger decreased 27 millions people stil...
215,237,ifrs group report new structure kpmg sa geneva...


In [19]:
docs = df_vector['summary'].values.tolist()


In [20]:
docs

['health promotion glossary world health organization geneva health promotion glossary whohprhep981 distr document may reviewed abstracted quoted reproduced translated part whole without prior written permission met useful purpose clarifying meaning relationship many terms common usage time conference adopted jakarta declaration leading health promotion 21st century number terms central jakarta declaration therefore included new version health promotion glossary definitions regarded final word terms included version glossary substantially different original definitions followed note explanation qualification bibliography lists sources referred directly text cross referencing intended improve understanding interrelationships different terms concepts health resource everyday life object living health regarded fundamental human right correspondingly people access basic resources health jakarta declaration leading health promotion 21st century july 1997 confirmed strategies action areas re

In [21]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [22]:
for sent in docs:
    average_vector = np.mean(np.array([get_embeddings(word) for word in nltk.word_tokenize(sent)]),axis=0)
    d = {sent:average_vector}
    out_dict.update(d)

In [23]:
import scipy

In [24]:

def get_similarity(query, doc):
    cos_sim = np.dot(query, doc)/(np.linalg.norm(query)*np.linalg.norm(doc))
    return cos_sim

In [25]:
def rank_text(query):
    query_vector = np.mean(np.array([get_embeddings(word) for word in nltk.word_tokenize(query)]),axis=0)
    rank = []
    for k,v in out_dict.items():
        rank.append((k,get_similarity(query_vector, v)))
    rank = sorted(rank, key=lambda x:x[1], reverse=True)
    return rank

In [31]:
top_10 = rank_text('how to audit water safety plans')
top_10[:10]

[('introduction 6 11 introduction 18 42 water supply networks urban rural areas turnkey projects 23 424 public safety considerations 32 5 public consultation disclosure requirements 56 71 unicef organization yemen intends carry wash program adverse consequences local environment well social sides local community every 10 minutes child dies preventable cases yemen however activities well coordinated term sustainability chapters presented sector nature impacts mitigation measures vary according type proposed works list grouped interventions interventions intervention health facilities includes a1 diarrhea treatment center dtc establishment operation including cleaning disinfection a5 fecal sludge management including septic tank cesspit r b unicef intervention following sectors b1 33 issued 2002 modified 2006 creation ministry water environment mwe comprised ground water surface water wastewater purification saline water desalination priorities water use drinking domestic use shall absol

In [32]:
for i in top_10:
  print('Title: ' + str(df[(df.summary == i[0])].Title.values[0]))
  print('Summary: ' + str(df[(df.summary == i[0])].summary.values[0]))
  print('URL: ' + str(df[(df.summary == i[0])].URL.values[0]) + '\n')

Title: ENVIRONMENTAL AND SOCIAL MANAGEMENT FRAMEWORK
Summary: introduction 6 11 introduction 18 42 water supply networks urban rural areas turnkey projects 23 424 public safety considerations 32 5 public consultation disclosure requirements 56 71 unicef organization yemen intends carry wash program adverse consequences local environment well social sides local community every 10 minutes child dies preventable cases yemen however activities well coordinated term sustainability chapters presented sector nature impacts mitigation measures vary according type proposed works list grouped interventions interventions intervention health facilities includes a1 diarrhea treatment center dtc establishment operation including cleaning disinfection a5 fecal sludge management including septic tank cesspit r b unicef intervention following sectors b1 33 issued 2002 modified 2006 creation ministry water environment mwe comprised ground water surface water wastewater purification saline water desalina